In [1]:
from IPython import display
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pretty_midi
import seaborn as sns
import tensorflow as tf

from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

# Tomb added
import random

In [ ]:
#Download Chorales
data_dir = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/chorales')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'midi',
      origin='https://github.com/jamesrobertlloyd/infinite-bach/tree/master/data/chorales/midi',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print(filenames)
print('Number of files:', len(filenames))

In [3]:
class UnsupportedMidiFileException(Exception):
  "Unsupported MIDI File"

In [4]:
"""Tomb made a pretty useful function"""
def check_pianoroll_dim(pianoroll):
        rows = len(pianoroll)  # This gives the number of rows
        columns = len(pianoroll[0])  # This assumes all rows have the same length

        print("Number of rows:", rows) # represents sequence length
        print("Number of columns:", columns) # represents the 4 octave range in midi 36-83
        print("Total dimesions of this pianoroll is", rows*columns)

In [5]:
def get_pianoroll(midi, nn_from, nn_thru, seqlen, tempo):
    pianoroll = midi.get_piano_roll(fs=2*tempo/60) # This is the core line which makes this matrix based on 8th note

    # print(f"piano_roll.shape[1] a.k.a song length!{pianoroll.shape[1]}")

    if pianoroll.shape[1] < seqlen:
        raise UnsupportedMidiFileException

    pianoroll = pianoroll[nn_from:nn_thru, 0:seqlen] # (48, 64) Pinoroll's value still NOT binary since it has velocity
    binary_pianoroll = np.heaviside(pianoroll, 0) # converting as a binary matrix
    transposed_pianoroll = np.transpose(binary_pianoroll) #(64, 48)
     
    # return binary_pianoroll
    return transposed_pianoroll # type numpy.ndarray

In [6]:
def read_midi(filename, sop_alto, seqlen):
  
  def add_rest_nodes(pianoroll):  # If all the elemets are zero, the rest node says 1, else 0
    rests = 1 - np.sum(pianoroll, axis=1)
    rests = np.expand_dims(rests, 1)
    return np.concatenate([pianoroll, rests], axis=1)
  
  
  # read midi file
  midi = pretty_midi.PrettyMIDI(filename)

  # An Exception error is thrown if there is a modulation(key change)
  if len(midi.key_signature_changes) !=1:
    raise UnsupportedMidiFileException

  # Modulate the given key to C major or C minor
  key_number = midi.key_signature_changes[0].key_number
  # transpose_to_c(midi, key_number)

  # Get Major key(keynode=0) or Minor key(keynode=1)
  keymode = np.array([int(key_number / 12)])

  # The Exception error thrown when tempo changes
  tempo_time, tempo = midi.get_tempo_changes()
  if len(tempo) != 1:
    raise UnsupportedMidiFileException
  if sop_alto:
    # The exception thrown if there are less than 2 parts
    if len(midi.instruments) < 2:
      raise UnsupportedMidiFileException
    # Get pianoRoll returns numpy.ndarray
    pr_s = get_pianoroll(midi.instruments[0], 36, 84, seqlen, tempo[0])
    pr_a = get_pianoroll(midi.instruments[1], 36, 84, seqlen, tempo[0])
    pr_b = get_pianoroll(midi.instruments[2], 36, 84, seqlen, tempo[0])
    
    
    sop_w_rest = add_rest_nodes(pr_s) 
    alt_w_rest = add_rest_nodes(pr_a)
    bass_w_rest = add_rest_nodes(pr_b)
    
    # return pr_s, pr_a, pr_b, keymode
    return sop_w_rest, alt_w_rest, bass_w_rest, keymode # All numpy.ndarray including keymode  

  else:
    #Get a pianoroll which gathered all the parts
    pr = get_pianoroll(midi, 36, 84, seqlen, tempo[0])
    return pr, keymode

In [27]:
"""Get the ingredients. 
Make the data(i.e manipulate the model as you tell it what you want) here for predict the 3rd note with given (x1,x2). 
Make list1 that has (xn, xn+1) pair elements 
and list2 which has (xn+2) elements"""

np.set_printoptions(threshold=np.inf) # Show the entire print, esp Matrix

x_all = [] #(1488960, 2) Total dim is 2977920.  1488960=495*64*47
y_all = [] 
keymodes = [] 
files = []

# n_notes = len(x_all) # 1488960. Not sure if it is correct
# print("n_notes!",n_notes)

# repeat the process with all the midi files
for file in glob.glob(str(data_dir/"**/*.mid*")):

  try:
    # make a window to get sequence pairs (Xn, Xn+1) -> Xn+2
    sops_data, alt, bass, keymode = read_midi(file, True, 64)
    for song in sops_data: # sops_data shape (64, 49)
      for i in range(len(song)-2): # range(0, 62) as song originally len 64
        input_sequence = song[i:i+2] # (Xn, Xn +1). print [0. 0.]  shape (2,)
        output_target = song[i+2] # Xn + 2. print 0.0  shape ()
        
        x_all.append(input_sequence)
        y_all.append(output_target)
  # throw exception for midi data which can not be used
  except UnsupportedMidiFileException:
    print("nah")


input_data = np.array(x_all) # shape (1488960, 2) Total dim is 2977920.  1488960=495*64*47. <class 'numpy.ndarray'>
output_data = np.array(y_all) # shape (1488960,). <class 'numpy.ndarray'>

#Reshape input data to (number_of_samples, 2, 64, 47)
#I made it 47 cz of the calculation 1488960=495*64*47 can only satisfy the total number while it makes sense but not sure why 47 instead of 49
# input_data = input_data.reshape(-1, 2, 64, 47) # shape (495, 2, 64, 47)
# # Reshape output data to (number_of_samples, 64, 47)
# output_data = output_data.reshape(-1, 64, 47) # shape (495, 64, 47)

i_train, i_test = train_test_split(range(len(input_data)),test_size=int(len(input_data)/2))
x_train = input_data[i_train] # shape (248, 2, 64, 47)
x_test = input_data[i_test] # shape (247, 2, 64, 47)
y_train = output_data[i_train] # shape (248, 64, 47)
y_test = output_data[i_test] # shape (247, 64, 47)

# x_train = x_train.reshape(-1, 2, 64 * 47)
# x_test = x_test.reshape(-1, 2, 64 * 47)

# y_train = y_train.reshape(-1, 64*47)
# y_test = y_test.reshape(-1, 64*47)

nah
nah
nah


#### Data preparation
-Convert to np.ndarray<br>
-Extract train and test data<br>
-Reshape(flatten) arrays to send to NNs<br>

In [34]:
input_data = np.array(x_all) # shape (1488960, 2) Total dim is 2977920.  1488960=495*64*47. <class 'numpy.ndarray'>
output_data = np.array(y_all) # shape (1488960,). <class 'numpy.ndarray'>

#Reshape input data to (number_of_samples, 2, 64, 47)
#I made it 47 cz of the calculation 1488960=495*64*47 can only satisfy the total number while it makes sense but not sure why 47 instead of 49
input_data = input_data.reshape(-1, 2, 64, 47) # shape (495, 2, 64, 47)
# Reshape output data to (number_of_samples, 64, 47)
output_data = output_data.reshape(-1, 64, 47) # shape (495, 64, 47)

i_train, i_test = train_test_split(range(len(input_data)),test_size=int(len(input_data)/2))
x_train = input_data[i_train] # shape (248, 2, 64, 47)
x_test = input_data[i_test] # shape (247, 2, 64, 47)
y_train = output_data[i_train] # shape (248, 64, 47)
y_test = output_data[i_test] # shape (247, 64, 47)

x_train = x_train.reshape(-1, 2, 64 * 47)
x_test = x_test.reshape(-1, 2, 64 * 47)

y_train = y_train.reshape(-1, 64*47)
y_test = y_test.reshape(-1, 64*47)

In [ ]:
# seq_length = x_train.shape[1] # 2 -> 時系列の長さ(時間方向の要素数)
# input_dim = x_train.shape[2] # 64 -> 入力の各要素の次元数
# output_dim = y_train.shape[2] # 47-> 出力の各要素の次元数

In [36]:
# Assuming the shapes you've provided: x_train, x_test, y_train, y_test

# Create an LSTM model
model = tf.keras.Sequential()

# Add an LSTM layer
model.add(tf.keras.layers.LSTM(units=64, input_shape=(2, 64*47)))  # LSTM units can be adjusted based on the complexity of the problem

# Add a dense layer for output
model.add(tf.keras.layers.Dense(64*47, activation='softmax'))  # Adjust the output shape based on your targets

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.evaluate(x_train, y_train, return_dict=True)

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))



8/8 [==============================] - 0s 2ms/step - loss: 512.5793 - accuracy: 0.0000e+00
Epoch 1/100
8/8 [==============================] - 1s 13ms/step - loss: 511.5201 - accuracy: 0.0000e+00 - val_loss: 508.7806 - val_accuracy: 0.0040
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 502.9782 - accuracy: 0.0000e+00 - val_loss: 493.0836 - val_accuracy: 0.0000e+00
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 480.9799 - accuracy: 0.0000e+00 - val_loss: 467.4860 - val_accuracy: 0.0000e+00
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 456.6798 - accuracy: 0.0000e+00 - val_loss: 449.9346 - val_accuracy: 0.0000e+00
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 443.2662 - accuracy: 0.0000e+00 - val_loss: 442.2012 - val_accuracy: 0.0000e+00
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 437.1392 - accuracy: 0.0000e+00 - val_loss: 438.6164 - val_accuracy: 0.0000e+00
Epoch 

KeyboardInterrupt: 

### Unused Code Below>>>

In [9]:
# """get inputs (Xn, Xn+1) and target (Xn+2) """
# def make_sequences(
#         dataset: tf.data.Dataset,
#         seq_length:int,
# ) -> tf.data.Dataset:
    
#     seq_length = seq_length+1

#     windows = dataset.window(seq_length, shift=1, stride=1, drop_remainder=True)
#     # for w in windows:
#     #    print(f"window!! {list(w.as_numpy_iterator())}")
    
#     flatten = lambda x: x.batch(seq_length, drop_remainder=True) # Assing lambda function to the variable "flatten"
#     sequences = windows.flat_map(flatten) # Flat_map falltens the "dataset of datasets" into a dataset of tensors

#     def split_labels(sequences):
#         inputs = sequences[:-1] # Could not see the inside as this func got in through map_func
#         output_dense = sequences[-1]
#         return inputs, output_dense
    
#     return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

In [10]:
# sop_tf_data = [] #  A list which contains (64 ,49) tf.daset elements
# sop, alt, bass, keymode = read_midi(f, True, 64)
# # sop = sop[0]
# # sop_tf = tf.data.Dataset.from_tensor_slices(sop)
# # print(list(sop_tf.as_numpy_iterator()))

# for i in sop:
#     sop_tf = tf.data.Dataset.from_tensor_slices(i)
#     sop_tf_data.append(sop_tf)

# # for t in sop_tf_data:
#     # print(list(t.as_numpy_iterator()))


In [11]:
# single_sop_tf = sop_tf_data[0]
# seq_length = 2
# # print(make_sequences(single_sop_tf, seq_length))

# seq_ds = make_sequences(single_sop_tf, seq_length) # 47 elements each input n output
# # type <'tensorflow _ParallelMapDataset'>
# # print(seq_ds.element_spec)
# # print(list(seq_ds.as_numpy_iterator())) # [(array([0., 0.]), 0.0), (array([0., 0.]), 0.0)...


# inputList = []
# targetList = []

# #<Check the dataset elements>
# for seq, target in seq_ds:
#   # print('Input sequence shape:', seq.shape) # shape (2,), a one-dimensional array (vector) with a length of 2.
#   # print('Input sequence elements:', seq[0: 1]) 
#   # print('target:', target) # shape ()
#   # print()
#   inputList.append(seq) # <class 'list'>
#   targetList.append(target) # <class 'list'>

# inputList = np.array(inputList) # <class 'numpy.ndarray'>. shape (47, 2), 2D matrix
# targetList = np.array(targetList) # <class 'numpy.ndarray'> shape (47,), 1D scalar